In [ ]:
import requests
import os
import json
from pprint import pprint
import warnings

In [ ]:
ZENODO_ACCESS_TOKEN = os.environ.get('ZENODO_ACCESS_TOKEN', '4WUTJLhMlKljZkknoc7llyMcTy3vjBy3I11j0mwUHVSYixxjt9Q72kLo8KMp')
ZENODO_URL = os.environ.get('ZENODO_URL', 'https://sandbox.zenodo.org')
params = {'access_token': ZENODO_ACCESS_TOKEN}
headers = {"Content-Type": "application/json"}

In [ ]:
publish_path='publish/1.11'
#Read metadata from zenodo.json file
with open(f'{publish_path}/zenodo.json', 'r') as f:
  zenodo_json = json.load(f)
zenodo_json

In [ ]:
# Create empty record
res = requests.post(
  f'{ZENODO_URL}/api/deposit/depositions',
  params=params,
  json={},
  headers=headers
)

assert (res.status_code == 201), f'Error creating record: {res.json()}'

newrecord = res.json()
newrecord_deposition_id = newrecord['id']
newrecord_bucket_url = newrecord["links"]["bucket"]
newversion_prereserve_doi = newrecord['metadata']['prereserve_doi']['doi']
pprint(newrecord)

In [ ]:
#Put metadata
res = requests.put(
  f'{ZENODO_URL}/api/deposit/depositions/{newrecord_deposition_id}',
  params = params,
  headers = headers,
  data = json.dumps(zenodo_json))

assert (res.status_code == 200), f'Error updating metadata {newrecord_deposition_id}: {res.json()}'


In [ ]:
#upload file
for file in zenodo_json['files']:
  filename = file['filename']
  with open(f'{publish_path}/{filename}', 'rb') as f:
    res = requests.put(
      f'{newrecord_bucket_url}/{filename}',
      params = params,
      data = f)
  assert (res.status_code == 201), f'Error uploading file {filename}: {res.json()}'
  assert (res.json()['size'] == file['size']), f'Error uploading file. Size mismatch: {res.json()['size']} != {file['size']}'
  assert (res.json()['checksum'] == file['checksum']), f'Error uploading file. Checksum mismatch: {res.json()['checksum']} != {file['checksum']}'
  #TODO: Verify MD5


In [ ]:
res

In [ ]:
res = requests.get(
  f'{ZENODO_URL}/api/deposit/depositions/{newrecord_deposition_id}',
  params = params,
  headers = headers
)
if res.status_code != 200:
  raise requests.RequestException(res.json())
res.json()

In [ ]:
res = requests.get(
  f'{ZENODO_URL}/api/deposit/depositions',
  params = params,
  headers = headers
)
if res.status_code != 200:
  raise requests.RequestException(res.json())
res.json()

# "New" Version 1.0

In [ ]:
publish_path='publish/1.0'
#Read metadata from zenodo.json file
with open(f'{publish_path}/zenodo.json', 'r') as f:
  zenodo_json = json.load(f)
zenodo_json

In [ ]:
#LATEST_DEPOSITION_ID = '129052'
LATEST_DEPOSITION_ID = '129592'
LATEST_DEPOSITION_ID = '130529'
# Create new version FROM record
#  f'{ZENODO_URL}/api/deposit/depositions/{LATEST_DEPOSITION_ID}/actions/newversion',
#  f'{ZENODO_URL}/api/records/{LATEST_DEPOSITION_ID}/versions',
res = requests.post(
  f'{ZENODO_URL}/api/deposit/depositions/{LATEST_DEPOSITION_ID}/actions/newversion',
  params = params,
  json = {},
  headers = headers
)
if res.status_code != 201:
  if res.status_code == 500 and res.json()['message'] == 'Misconfigured search.':
    warnings.warn(f'Error creating new version for deposition id {LATEST_DEPOSITION_ID}: {res.json()}')
  else:
    assert (res.status_code == 201), f'Error creating new version for deposition id {LATEST_DEPOSITION_ID}: {res.json()}'



In [ ]:
res.json()

In [ ]:
# TO BE KEEP in case temporla issues appear again
#TEMP_ID = 129747
#res = requests.get(
#  f'{ZENODO_URL}/api/deposit/depositions/{TEMP_ID}',
#  params = params,
#  headers = headers
#)
#assert (res.status_code == 200), f'Error retriving new verions deposition {TEMP_ID}: {res.json()}'

newversion_record = res.json()
newversion_deposition_id = newversion_record['id']
newversion_bucket_url = newversion_record['links']['bucket']
newversion_prereserve_doi = newversion_record['metadata']['prereserve_doi']['doi']
newversion_record

In [ ]:
res.json()

In [ ]:
#delete files
for file in newversion_record['files']:
  res = requests.delete(
    f'{ZENODO_URL}/api/deposit/depositions/{newversion_deposition_id}/files/{file['id']}',
    params = params,
  )
  assert (res.status_code == 204), f'Error deleting file {file['filename']}: {res.json()}'


In [ ]:
#Put metadata
res = requests.put(
  f'{ZENODO_URL}/api/deposit/depositions/{newversion_deposition_id}',
  params = params,
  headers = headers,
  data = json.dumps(zenodo_json))

assert (res.status_code == 200), f'Error updating metadata {newversion_deposition_id}: {res.json()}'


In [ ]:
#upload file
for file in zenodo_json['files']:
  filename = file['filename']
  with open(f'{publish_path}/{filename}', 'rb') as f:
    res = requests.put(
      f'{newversion_bucket_url}/{filename}',
      params = params,
      data = f)
  assert (res.status_code == 201), f'Error uploading file {filename}: {res.json()}'
  assert (res.json()['size'] == file['size']), f'Error uploading file. Size mismatch: {res.json()['size']} != {file['size']}'
  assert (res.json()['checksum'] == file['checksum']), f'Error uploading file. Checksum mismatch: {res.json()['checksum']} != {file['checksum']}'


# Parking Code

In [ ]:
#Read metadata
with open('publish/1.11/metadata.json', 'r') as f:
  d = json.load(f)
json.dumps(d)
#Put metadata
r = requests.put(
  f'{ZENODO_URL}/api/deposit/depositions/{newversion_deposition_id}',
  params = params,
  headers = headers,
  data = json.dumps(d))
r.status_code

In [ ]:
#upload file
filename = 'cf-conventions.pdf'
with open(f'publish/1.11/{filename}', 'rb') as f:
  r = requests.put(
    f'{newversion_bucket_url}/{filename}',
    params = params,
    data = f)
  #TODO: Verify MD5
  if r.status_code != 201:
    raise Exception(r.json())

# New Version 1.12-rc7

In [ ]:
LATEST_DEPOSITION_ID = '129052'
# Create new record
r = requests.post(f'{ZENODO_URL}/api/deposit/depositions/{LATEST_DEPOSITION_ID}/actions/newversion',
                   params=params,
                   json={},
                   headers=headers)
# 201
if r.status_code == 201:
  newversion_record = r.json()
  newversion_deposition_id = newversion_record['id']
  newversion_bucket_url = newversion_record['links']['bucket']
  newversion_prereserve_doi = newversion_record['metadata']['prereserve_doi']['doi']
else:
  raise requests.RequestException(r.json())



In [ ]:
#delete files
for file in newversion_record['files']:
  r = requests.delete(
    f'{ZENODO_URL}/api/deposit/depositions/{newversion_deposition_id}/files/{file['id']}',
    params = params,
  )
  if r.status_code != 204:
    r.json()

In [ ]:
#Read metadata
with open('publish/1.12-rc7/metadata.json', 'r') as f:
  d = json.load(f)
json.dumps(d)


In [ ]:
#Put metadata
r = requests.put(
  f'{ZENODO_URL}/api/deposit/depositions/{newversion_deposition_id}',
  params = params,
  headers = headers,
  data = json.dumps(d))
if r.status_code != 200:
    r.json()

In [ ]:
#upload file
filename = 'cf-conventions.pdf'
with open(f'publish/1.12-rc7/{filename}', 'rb') as f:
  r = requests.put(
    f'{newversion_bucket_url}/{filename}',
    params = params,
    data = f)
  #TODO: Verify MD5
  if r.status_code != 201:
    raise Exception(r.json())

In [ ]:
newversion_record